## Implementation of PilotNet, with variation

Implement [PilotNet](https://arxiv.org/pdf/1604.07316.pdf) using Keras (with theano backend), with various differences

In [1]:
from theano.sandbox import cuda
cuda.use('gpu0')

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 1080 (CNMeM is enabled with initial size: 90.0% of memory, cuDNN 5110)
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29



In [2]:
%matplotlib inline
import utils; reload(utils)
from utils import *

Using Theano backend.


### Check data format

train using udacity data

check the csv output

In [3]:
df = pd.read_csv("data/driving_log.csv", names=['Center', 'Left', 'Right', 'Steering Angle', 'Throttle', 'Break', 'Speed'])
df.head(3)

,Center,Left,Right,Steering Angle,Throttle,Break,Speed
0,data/IMG/center_2017_04_06_17_06_56_921.jpg,data/IMG/left_2017_04_06_17_06_56_921.jpg,data/IMG/right_2017_04_06_17_06_56_921.jpg,0.0,0.0,0.0,0.000002
1,data/IMG/center_2017_04_06_17_06_57_009.jpg,data/IMG/left_2017_04_06_17_06_57_009.jpg,data/IMG/right_2017_04_06_17_06_57_009.jpg,0.0,0.0,0.0,0.000007
2,data/IMG/center_2017_04_06_17_06_57_085.jpg,data/IMG/left_2017_04_06_17_06_57_085.jpg,data/IMG/right_2017_04_06_17_06_57_085.jpg,0.0,0.0,0.0,0.000002


###  First, only predict steering angle

define input and output, and load to memory.

In [4]:
X_train = np.array([img_path_to_arr(p) for p in df['Center']])
y_train = df['Steering Angle'].values

input_shape = X_train.shape[1:]

In [5]:
def get_model_original():
    model = Sequential()

    model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=input_shape, output_shape=input_shape))
#     model.add(Cropping2D(cropping=((50,20), (0,0)), input_shape=input_shape))
    
    model.add(Conv2D(24, kernel_size=(5,5), strides=(2,2), activation='relu'))
    
    model.add(Conv2D(36, kernel_size=(5,5), strides=(2,2), activation='relu'))
    
    model.add(Conv2D(48, kernel_size=(5,5), strides=(2,2), activation='relu'))

    model.add(Conv2D(64, kernel_size=(3,3), strides=(1,1), activation='relu'))

    model.add(Conv2D(64, kernel_size=(3,3), strides=(1,1), activation='relu'))

    model.add(Flatten())    

    model.add(Dense(100, activation='relu'))
    
    model.add(Dense(50, activation='relu'))
    
    model.add(Dense(10, activation='relu'))

    model.add(Dense(1))

    model.compile(optimizer="adam", loss="mse") 
    model.summary()
    return model

In [6]:
model_original = get_model_original()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 3, 160, 320)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 78, 158)       1824      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 36, 37, 77)        21636     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 48, 17, 37)        43248     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 64, 15, 35)        27712     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 64, 13, 33)        36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 27456)             0         
__________

In [7]:
model_original.fit(X_train, y_train, validation_split = 0.2, shuffle = True, epochs = 5)

Train on 9742 samples, validate on 2436 samples
Epoch 1/5
9742/9742 [==============================] - 8s - loss: 0.0228 - val_loss: 0.0111
Epoch 2/5
9742/9742 [==============================] - 8s - loss: 0.0179 - val_loss: 0.0090
Epoch 3/5
9742/9742 [==============================] - 8s - loss: 0.0162 - val_loss: 0.0082
Epoch 4/5
9742/9742 [==============================] - 8s - loss: 0.0152 - val_loss: 0.0112
Epoch 5/5
9742/9742 [==============================] - 8s - loss: 0.0139 - val_loss: 0.0118
